In [1]:
import numpy as np
from copy import deepcopy
from dataclasses import dataclass , field
from typing import Any, Literal , Optional

PROB_TYPE = Literal['linprog' , 'quadprog' , 'socp']

def rescale(v : np.ndarray , scale_value : float = 1):
    return scale_value * v / (v.sum() + 1e-6)

@dataclass
class SolverInput:
    u : np.ndarray
    lin_con : 'LinearConstraint'
    bnd_con : 'BoundConstraint'
    turn_con : Optional['TurnConstraint'] = None
    cov_con  : Optional['CovConstraint'] = None
    w0 : np.ndarray | Any = None
    wb : np.ndarray | Any = None
    u_scaler : float = 1.

    def __post_init__(self):
        self.check()

    def check(self):
        assert self.u.ndim == 1
        N = len(self.u)
        self.lin_con.check(N)
        self.bnd_con.check(N)
        if self.turn_con: self.turn_con.check(N)
        if self.cov_con: self.cov_con.check(N)
        assert self.w0 is None or self.w0.shape == (N , )
        assert self.wb is None or self.wb.shape == (N , )

    def copy(self): return deepcopy(self)

    def rescale(self , inplace = False):
        new = self if inplace else self.copy()
        new.u_scaler = new.u.std() + 1e-6
        new.u /= new.u_scaler
        if new.cov_con:  new.cov_con.rescale(new.u_scaler)
        if new.turn_con: new.turn_con.rescale(new.u_scaler)
        return new
    
    def utility(self , w : np.ndarray , prob_type : PROB_TYPE = 'linprog' ,
                turn = True ,  qobj = True):
        value = w.dot(self.u)
        if qobj and prob_type != 'linprog' and self.cov_con is not None and self.wb is not None:
            value -= 0.5 * self.cov_con.lmbd * self.cov_con.variance(w - self.wb)
        if turn and self.turn_con is not None and self.w0 is not None:
            value -= self.turn_con.rho * np.abs(w - self.w0).sum()

        return value * self.u_scaler
    
    def optim_accuracy(self , w : np.ndarray):
        lin_ub_bias = np.min(self.lin_con.ub - self.lin_con.A.dot(w))
        lin_lb_bias = np.min(self.lin_con.A.dot(w) - self.lin_con.lb)

        bnd_ub_bias = np.min(self.bnd_con.ub - w)
        bnd_lb_bias = np.min(w - self.bnd_con.lb)
        
        excess_turn = 0.0 if self.turn_con is None else self.turn_con.to - np.abs(w - self.w0).sum()
        if self.cov_con is not None and self.cov_con.te is not None:
            optimize_te = np.sqrt(self.cov_con.variance(w - self.wb))
            excess_te = self.cov_con.te - optimize_te
        else:
            excess_te = 0.0

        cond_expr = lambda x : ('(√)' if x >= -1e-6 else '(X)') + str(x)
        accuracy = {
            'lin_ub_bias' : cond_expr(lin_ub_bias) ,
            'lin_lb_bias' : cond_expr(lin_lb_bias) ,
            'bnd_ub_bias' : cond_expr(bnd_ub_bias) , 
            'bnd_lb_bias' : cond_expr(bnd_lb_bias) ,
            'excess_turn' : cond_expr(excess_turn) ,
            'excess_te'   : cond_expr(excess_te) ,
        }
        return accuracy

    @classmethod
    def rand(cls , N : int = 3 , with_turn = True , with_cov = True):
        return cls(
            u = np.random.randn(N) ,
            lin_con = LinearConstraint.rand(N) ,
            bnd_con = BoundConstraint.rand(N) ,
            turn_con = TurnConstraint.rand(N) if with_turn else None ,
            cov_con = CovConstraint.rand(N) if with_cov else None ,
            w0 = rescale(np.random.rand(N)) ,
            wb = rescale(np.random.rand(N)) ,
        )

@dataclass
class LinearConstraint:
    '''
    A   : exposure matrix
    type: str, ra(w<=ub & w>=lb)、lo(w>=lb)、up(w<=ub)、fx(w=lb&w=lb)四种。
    lb  : float, lower bound
    ub  : float, upper bound
    '''
    A : np.ndarray
    type : np.ndarray
    lb : np.ndarray
    ub : np.ndarray

    def __bool__(self): return True

    def check(self , N : int):
        assert self.A.ndim == 2 and self.A.shape[1] == N , self.A
        K = self.A.shape[0]
        assert self.type.shape == (K, ) and self.ub.shape == (K, ) and self.lb.shape == (K, )
        assert np.isin(self.type , ('ra', 'lo', 'up', 'fx')).all() , np.setdiff1d(self.type , ('ra', 'lo', 'up', 'fx'))
        assert (self.lb <= self.ub).all() , np.where(self.lb > self.ub)[0]

    @classmethod
    def rand(cls , N : int , L = 1):
        return cls(
            A = np.concatenate([np.ones((1,N)) , np.random.rand(L , N)] , axis=0) ,
            type = np.concatenate([['fx'] , np.repeat(['ra'] , L)]),
            lb = np.concatenate([[1.] , np.random.rand(L)]),
            ub = np.ones(1 + L),
        )

@dataclass
class BoundConstraint:
    '''
    type: str, ra(w<=ub & w>=lb)、lo(w>=lb)、up(w<=ub)、fx(w=lb&w=lb)四种。
    lb  : float, lower bound
    ub  : float, upper bound
    '''
    type : np.ndarray
    lb : np.ndarray
    ub : np.ndarray

    def __bool__(self): return True

    def check(self , N : int):
        assert self.type.shape == (N, ) and self.ub.shape == (N, ) and self.lb.shape == (N, )
        assert np.isin(self.type , ('ra', 'lo', 'up', 'fx')).all() , np.setdiff1d(self.type , ('ra', 'lo', 'up', 'fx'))
        assert (self.lb <= self.ub).all() , np.where(self.lb > self.ub)[0]

    @classmethod
    def rand(cls , N : int):
        return cls(
            type = np.repeat(['ra'] , N) ,
            lb = np.zeros(N),
            ub = np.ones(N),
        )

@dataclass
class TurnConstraint:
    '''
    to : float, double side constraint
    rho: float, penalty factor in objective function
    '''
    to  : float = 0.2
    rho : float = 0.00001

    def __bool__(self): return True

    def check(self , N : int):
        assert self.rho >= 0.0 , self.rho
        assert self.to > 0.0 , self.to

    def rescale(self , u_scale : float):
        self.rho = self.rho / u_scale
        return self

    @classmethod
    def rand(cls , N : int): return cls(to = 1.5)

@dataclass
class CovConstraint:
    '''
    lmbd: float, risk aversion
    te  : float, tracking error constraint

    input type 1
    F: (L , N) array, common factor exposure
    C: (L , L) array, common factor covariance
    S: (N ,)  array, specific risk (can be None)

    input type 2
    cov: (N , N) array, instrument covariance
    '''
    lmbd : float = 0.5
    te   : Optional[float] = None
    F : np.ndarray | Any = None
    C : np.ndarray | Any = None
    S : np.ndarray | Any = None
    cov : np.ndarray | Any = None
    cov_type : Literal['normal' , 'model'] = 'model'
    
    def __bool__(self): return True

    def check(self, N : int):
        assert self.lmbd > 0 , self.lmbd
        if self.te is not None: assert self.te > 0.
        if self.cov_type == 'model':
            assert self.F.ndim == 2 and self.C.ndim == 2 , (self.F.shape , self.C.shape)
            L = self.F.shape[0]
            assert self.F.shape == (L , N) and self.C.shape == (L, L)
            assert self.S is None or (self.S.shape == (N,) and (self.S >= 0.0).all()) , self.S.shape
        else:
            assert self.cov.shape == (N, N)

    def rescale(self , u_scale : float):
        if self.cov_type == 'model':
            cov_scale = np.diagonal(self.C).mean() + 1e-6
            if self.S is not None: 
                cov_scale = np.sqrt(cov_scale * (self.S.mean() + 1e-6)) # geometric mean
                self.S = self.S / cov_scale
            self.C = self.C / cov_scale
        else:
            cov_scale = np.diagonal(self.cov).mean() + 1e-6
            self.cov  = self.cov / cov_scale
        self.lmbd = self.lmbd / u_scale * cov_scale
        if self.te is not None: self.te = self.te / np.sqrt(cov_scale)

    def variance(self , w : np.ndarray | Any):
        if self.cov_type == 'model':
            quad_term = w.T.dot(self.F.T).dot(self.C).dot(self.F).dot(w) 
            if self.S is not None: quad_term += (w * self.S).dot(w)
        else:
            quad_term = w.T.dot(self.cov).dot(w)
        return quad_term

    @classmethod
    def rand(cls , N : int , cov_type : Literal['normal' , 'model'] = 'model'):
        if cov_type == 'normal':
            v = np.random.randn(N , 10)
            cov = v @ v.T + np.eye(N) * 1e-6
            return cls(cov = cov , te = 1. , cov_type = cov_type)
        else:
            L : int = max(2 , N // 5)
            F = np.random.randn(L , N)
            v = np.random.randn(L , N)
            C = v @ v.T + np.eye(L) * 1e-6
            v = np.random.randn(N , 2 * N)
            S = v.std(axis = 1)
            return cls(F = F , C = C , S = S , te = 1. , cov_type = cov_type)


In [2]:
from dataclasses import dataclass
from typing import Any , Callable , ClassVar , Literal

import src.factor.optimizer as optimzer

@dataclass
class SolverParam:
    prob_type   : PROB_TYPE = 'linprog'
    engine_type : Literal['mosek' , 'cvxopt' , 'cvxpy'] = 'mosek'
    solver_type : Literal['mosek' , 'ecos' , 'osqp' , 'scs'] = 'ecos' # only in cvxpy

    DEFAULT_SOLVER_PARAM : ClassVar[dict[str,dict]] = {
        'cvxpy.mosek': {'solver': 'MOSEK','max_iters': 200, 'bnd_inf': 100.0},
        'cvxpy.ecos': {'solver': 'ECOS','max_iters': 200, 'bnd_inf': 100.0},
        'cvxpy.osqp': {'solver': 'OSQP','bnd_inf': 100.0},
        'cvxpy.scs': {'solver': 'SCS','eps': 1e-6, 'bnd_inf': 100.0},
        'mosek': {},
        'cvxopt': {'show_progress': False}
    }

    def __post_init__(self):
        key = f'{self.engine_type}.{self.solver_type}' if self.engine_type == 'cvxpy' else self.engine_type
        self.param = self.DEFAULT_SOLVER_PARAM[key]

    def select_solver(self) -> Callable:
        solve = getattr(getattr(getattr(optimzer , f'{self.engine_type}_solver') , self.prob_type) , 'solve')
        assert callable(solve)
        return solve


In [7]:
import numpy as np
import mosek

class Solver:
    pinf = 0.0
    ninf = 0.0

    def __init__(self , input : SolverInput , 
                 prob_type : PROB_TYPE , 
                 solver_param : dict = {}):
        self.input = input
        self.prob_type : PROB_TYPE = prob_type
        self.solver_param = solver_param
        self.parse()

    def parse(self):
        self.u  = self.input.u
        self.w0 = self.input.w0
        self.wb = self.input.wb

        self.bnd_type = self.from_str_to_mosek_bnd_key(self.input.bnd_con.type)
        self.bnd_lb   = self.input.bnd_con.lb
        self.bnd_ub   = self.input.bnd_con.ub

        self.lin_A    = self.input.lin_con.A
        self.lin_type = self.from_str_to_mosek_bnd_key(self.input.lin_con.type)
        self.lin_lb   = self.input.lin_con.lb
        self.lin_ub   = self.input.lin_con.ub

        if self.prob_type != 'linprog' and self.input.cov_con is not None and self.wb is not None:
            self.cov_type   = self.input.cov_con.cov_type
            self.lmbd       = self.input.cov_con.lmbd
            self.cov        = self.input.cov_con.cov
            self.F          = self.input.cov_con.F
            self.C          = self.input.cov_con.C
            self.S          = self.input.cov_con.S
            self.te         = self.input.cov_con.te
        else:
            self.cov_type   = None
            self.te         = None

        if self.input.turn_con is not None and self.w0 is not None:
            self.turn_type = 'double'
            self.to  = self.input.turn_con.to
            self.rho = self.input.turn_con.rho
        else:
            self.turn_type = None

        return self

    def solve(self , turn = True , qobj = True , qcon = True):
        # variable sequence:
        # num_N , num_T (0 or num_N) , num_L (0 or len(self.F)) , num_Q (0 or 2)
        num_N = len(self.u)
        num_T = 0 if not turn or self.turn_type is None else num_N
        num_L = 0 if (not qobj and not qcon) or self.cov_type != 'model' else len(self.F)
        num_Q = 0 if not qcon or self.te is None else 2

        with mosek.Env() as env:
            with env.Task() as task:
                # setting
                self.task_init(task)
                # num_N stock weight
                self.task_addvars(task , num_N , self.bnd_type , self.bnd_lb , self.bnd_ub , -self.u)
                # num_N absolute turnover
                self.task_addvars(task , num_T , mosek.boundkey.lo , 0.0 , self.pinf , self.rho)
                # num_L factor exposure
                self.task_addvars(task , num_L , mosek.boundkey.fr , self.ninf , self.pinf , 0)
                # num_Q factor transformation
                self.task_addvars(task , num_Q , mosek.boundkey.lo , 0.0 , self.pinf , 0)

                # quad objective
                self.task_add_quad_obj(task , qobj , num_N , num_N + num_T)
                
                # lin constraints
                self.task_addlcons(task , len(self.lin_A) , self.lin_type , self.lin_lb , self.lin_ub , 
                                   np.arange(num_N).reshape(1,-1).repeat(len(self.lin_A),0), self.lin_A)
                # turnover constraint
                self.task_add_turn_con(task , turn , num_N , num_N)
                # L factor equivalent to F.dot(w)
                self.task_add_quad_model_con(task , qobj or qcon , num_N , num_N + num_T)
                # Quad TE constraint
                self.task_add_quad_te_con(task , qcon , num_N , num_N + num_T , num_N + num_T + num_L)
                
                # perform
                task.putobjsense(mosek.objsense.minimize)
                trmcode = task.optimize()
                task.solutionsummary(mosek.streamtype.msg)
                sol_sta = task.getsolsta(mosek.soltype.itr)
                xx = [0.0] * task.getnumvar()
                task.getxx(mosek.soltype.itr, xx)

                if sol_sta == mosek.solsta.optimal:
                    is_success = True
                    status = 'optimal'
                elif sol_sta == mosek.solsta.unknown and trmcode == mosek.rescode.trm_max_iterations:
                    is_success = True
                    status = 'max_iteration'
                elif sol_sta == mosek.solsta.unknown and trmcode == mosek.rescode.trm_stall:
                    is_success = True
                    status = 'stall'
                else:
                    is_success = False
                    status = ''

                self.task = task
        w = np.array(xx[:num_N])
        if is_success: 
            self.optimal_x = xx
            self.optimal_w = w
            self.turn = turn
            self.qobj = qobj
            self.qcon = qcon
        return w, is_success, status
    
    def task_init(self , task : mosek.Task):
        [task.putparam(key, val) for key, val in self.solver_param.items()]

    def enum(self , num : int , *args):
        if num <= 0: return []
        g = lambda x,i:(x[i] if isinstance(x , (np.ndarray , list)) else x)
        return [(i , [g(arg , i) for arg in args]) for i in range(num)]
    
    def task_addvars(self , task : mosek.Task , num : int ,
                     bound_key : np.ndarray | Any , bound_lb : np.ndarray | Any , bound_ub : np.ndarray | Any ,
                     coef_obj : np.ndarray | Any = 0.):
        if num <= 0: return
        start = task.getnumvar()
        task.appendvars(num)
        for j , (bkx , blx , bux , cj) in self.enum(num , bound_key , bound_lb , bound_ub , coef_obj):
            task.putvarbound(j + start, bkx , blx , bux)
            if cj is not None: task.putcj(j + start, cj)
    
    def task_add_quad_obj(self , task : mosek.Task , cond : bool , N : int , start_of_L : int):
        if self.cov_type is None or not cond: return

        if self.cov_type == 'normal':
            u = self.u.T + self.lmbd * self.wb.dot(self.cov)
            idx    = np.tril_indices(N)
            qosubi = idx[0]
            qosubj = idx[1]
            qoval  = self.lmbd * self.cov[idx]
        else:
            u = self.u.T + self.lmbd * (self.wb.dot(self.F.T).dot(self.C).dot(self.F) + (0 if self.S is None else self.wb * self.S))
            idx = np.tril_indices(len(self.F))
            if self.S is None:
                qosubi = idx[0] + start_of_L
                qosubj = idx[1] + start_of_L
                qoval  = self.lmbd * self.C[idx]
            else: 
                qosubi = np.concatenate([np.arange(N) , idx[0] + start_of_L])
                qosubj = np.concatenate([np.arange(N) , idx[1] + start_of_L])
                qoval  = self.lmbd * np.concatenate([self.S , self.C[idx]])
        # override linear objective coefficient
        [task.putcj(j, u[j]) for j in range(N)]
        # add quad objective
        task.putqobj(qosubi, qosubj, qoval)

    def task_addlcons(self , task : mosek.Task , num : int ,
                      bound_key : np.ndarray | Any , bound_lb : np.ndarray | Any , bound_ub : np.ndarray | Any ,
                      lcon_sub : np.ndarray | list , lcon_val : np.ndarray | list):
        start = task.getnumcon()
        task.appendcons(num)
        con_iter = self.enum(num , bound_key , bound_lb , bound_ub , lcon_sub , lcon_val)
        for i , (bkx , blx , bux , subi , vali) in con_iter:
            task.putconbound(i + start, bkx , blx , bux)
            task.putarow(i + start, subi , vali)

    def task_add_turn_con(self , task : mosek.Task , cond : bool , N : int , start_of_T : int):
        if not cond or self.turn_type is None: return
        # 1 : total turnover constraint
        self.task_addlcons(task , 1 , mosek.boundkey.up , self.ninf , self.to , [start_of_T + np.arange(N)] , [np.ones(N)])
        # N : turnover contrains w1 - delta <= w0
        self.task_addlcons(task , N , mosek.boundkey.up , self.ninf , self.w0 , 
                           [[i , start_of_T + i] for i in range(N)] , [[1. , -1.]] * N)
        # N : turnover contrains -w1 - delta <= -w0
        self.task_addlcons(task , N , mosek.boundkey.up , self.ninf , -self.w0 , 
                           [[i , start_of_T + i] for i in range(N)] , [[-1. , -1.]] * N)
        
    def task_add_quad_model_con(self , task : mosek.Task , cond : bool , N : int , start_of_L : int):
        if not cond or self.cov_type != 'model': return
        # num_L factors == self.F[i,:].dot(num_N variables)
        num_L = len(self.F)
        self.task_addlcons(task , num_L , mosek.boundkey.fx , 0.0, 0.0 , 
                           np.concatenate([np.arange(N).reshape(1,-1).repeat(num_L,0), 
                                            start_of_L + np.arange(num_L).reshape(-1,1)],axis=1) ,
                           np.concatenate([self.F,-np.ones((num_L,1))],axis=1))
        
    def task_add_quad_te_con(self , task : mosek.Task , cond : bool , N : int , start_of_L : int , start_of_Q : int):
        if not cond or self.cov_type is None or self.te is None: return
        te_sq = self.te ** 2
        start = task.getnumcon()

        if self.cov_type == 'normal': 
            idx  = np.tril_indices(N)
            qcub = 0.5 * te_sq - 0.5 * self.wb.dot(self.cov).dot(self.wb)

            task.appendcons(1)
            task.putconbound(start, mosek.boundkey.up, self.ninf, qcub)
            task.putarow(start, np.arange(N), -self.wb.dot(self.cov))
            task.putqconk(start, idx[0], idx[1], self.cov[idx])
            
        elif self.cov_type == 'model': 
            # total risk  
            task.appendcons(3) 
            task.putconbound(start, mosek.boundkey.up, self.ninf, te_sq)
            task.putarow(start, [start_of_Q , start_of_Q + 1], [1.0, 1.0])

            # common risk
            idx = np.tril_indices(len(self.F))
            qcub  = -0.5 * self.F.dot(self.wb).T.dot(self.C).dot(self.F.dot(self.wb))
            qcsub = np.concatenate([start_of_L + np.arange(len(self.F)) , [start_of_Q]])
            qcval = np.concatenate([-self.F.dot(self.wb).T.dot(self.C) , [-0.5]])

            task.putconbound(start + 1, mosek.boundkey.up, self.ninf, qcub)
            task.putarow(start + 1, qcsub, qcval)
            task.putqconk(start + 1, start_of_L + idx[0], start_of_L + idx[1] , self.C[idx])
            
            # spec risk
            qcub  = -0.5 * (self.wb * self.S).dot(self.wb)
            qcsub = np.concatenate([np.arange(N) , [start_of_Q + 1]])
            qcval = np.concatenate([-self.wb * self.S , [-0.5]])

            task.putconbound(start + 2, mosek.boundkey.up, self.ninf, qcub)
            task.putarow(start + 2, qcsub, qcval)
            task.putqconk(start + 2, np.arange(N) , np.arange(N) , self.S)
    
    def utility(self , w : Optional[np.ndarray] = None , 
                prob_type : Optional[PROB_TYPE] = None ,
                turn : Optional[bool] = None , 
                qobj : Optional[bool] = None):
        if w is None: w = self.optimal_w
        if prob_type is None: prob_type = self.prob_type
        if turn is None: turn = self.turn
        if qobj is None: qobj = self.qobj
        return self.input.utility(w , prob_type , turn , qobj)
    
    def optim_accuracy(self , w : Optional[np.ndarray] = None):
        if w is None: w = self.optimal_w
        return self.input.optim_accuracy(w)

    @classmethod
    def from_str_to_mosek_bnd_key(cls , s : str | list | np.ndarray):
        if isinstance(s , (list , np.ndarray)):
            return [cls.from_str_to_mosek_bnd_key(k) for k in s]
        else:
            if s == 'fx': return mosek.boundkey.fx
            elif s == 'lo': return mosek.boundkey.lo
            elif s == 'up': return mosek.boundkey.up
            elif s == 'ra': return mosek.boundkey.ra
            else: raise KeyError(s)

In [12]:
from src.factor.optimizer.input.util import SolverInput
from src.factor.optimizer.api import PortOptimizer
input = SolverInput.rand(3)
solver_params = SolverParam('socp')
input1 = input.rescale()

s = Solver(input , solver_params.prob_type , solver_params.param)
t = Solver(input1 , solver_params.prob_type , solver_params.param)

In [13]:
s.solve() , t.solve()

quad_obj here
turn_con here
model_con here
te_con here
quad_obj here
turn_con here
model_con here
te_con here


((array([0.51948187, 0.42763787, 0.05288026]), True, 'optimal'),
 (array([0.51948239, 0.4276374 , 0.05288021]), True, 'optimal'))

In [6]:
s.utility() , t.utility()

(0.4746144515690028, 0.47461053588686475)

In [307]:
s.optim_accuracy() , t.optim_accuracy()

({'lin_ub_bias': '(√)0.07311444718306537',
  'lin_lb_bias': '(√)0.572515514135601',
  'bnd_ub_bias': '(√)1.784292713580271e-09',
  'bnd_lb_bias': '(√)0.01885494088967974',
  'excess_turn': '(√)3.84564113886654e-09',
  'excess_te': '(√)0.16848890159580487'},
 {'lin_ub_bias': '(√)0.07311444049087357',
  'lin_lb_bias': '(√)0.5725155156104073',
  'bnd_ub_bias': '(√)8.502418324951577e-09',
  'bnd_lb_bias': '(√)0.018854947608708237',
  'excess_turn': '(√)6.913621231063871e-09',
  'excess_te': '(√)0.10524054001211358'})

In [5]:
from src.factor.optimizer.input.util import SolverInput
from src.factor.optimizer.api import PortOptimizer

input = SolverInput.rand(3)
optim = PortOptimizer('socp')
input1 = input.rescale()
input1.cov_con.model_to_normal()

s = optim.solve(input , True)
t = optim.solve(input1 , True)

s , t


((array([0.13041328, 0.13989465, 0.72969208]),
  True,
  'optimal',
  -13.582736622086584,
  {'lin_ub_bias': '(√)0.0',
   'lin_lb_bias': '(√)0.0',
   'bnd_ub_bias': '(√)0.2703079205673595',
   'bnd_lb_bias': '(√)0.13041327508268075',
   'excess_turn': '(√)0.30463993199996997',
   'excess_te': '(√)0.24589367758549685'}),
 (array([0.13041327, 0.13989464, 0.72969208]),
  True,
  'optimal',
  -13.582697681962049,
  {'lin_ub_bias': '(√)0.0',
   'lin_lb_bias': '(√)0.0',
   'bnd_ub_bias': '(√)0.2703079172833218',
   'bnd_lb_bias': '(√)0.13041327266385142',
   'excess_turn': '(√)0.30463992543189455',
   'excess_te': '(√)0.21619307804462817'}))

In [7]:
input.cov_con , input1.cov_con

(CovConstraint(lmbd=200.0, te=1.0, F=array([[ 0.03790486,  1.37002184,  0.16496559],
        [-0.6940561 ,  0.01503058,  0.00951563]]), C=array([[0.31950983, 0.79858796],
        [0.79858796, 3.03345401]]), S=array([1.07347212, 0.63799947, 1.01064067]), cov=None, cov_type='model'),
 CovConstraint(lmbd=956.5543436247365, te=0.9004381824152431, F=array([[ 0.03790486,  1.37002184,  0.16496559],
        [-0.6940561 ,  0.01503058,  0.00951563]]), C=array([[0.25905503, 0.64748627],
        [0.64748627, 2.4594909 ]]), S=array([0.8703593 , 0.51728291, 0.81941626]), cov=array([[ 2.02143412, -0.6275124 , -0.08852419],
        [-0.6275124 ,  1.03074081,  0.06894638],
        [-0.08852419,  0.06894638,  0.82872158]]), cov_type='normal'))